In [3]:
import numpy as np

## Generating random distance

In [4]:
def generate_distances(num_cities):
    distances = np.zeros(shape=(num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j and distances[i, j] == 0:
                distances[i, j] = distances[j, i] = np.random.randint(3, 51)
    return distances

In [21]:
class AntColony:
    def __init__(self, distances, num_ants, num_iterations, evaporation_rate=0.5, alpha=1, beta=2):
        self.distances = distances
        self.num_cities = len(distances)
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromones = np.ones_like(distances)
    
    def run(self):
        shortest_path = None
        shortest_distance = float('inf')

        for _ in range(self.num_iterations):
            paths = self.generate_paths()
            self.update_pheromones(paths)
            for path in paths:
                path.append(path[0])
                distance = self.calculate_distance(path)
                if distance < shortest_distance:
                    shortest_distance = distance
                    shortest_path = path
        return shortest_path, shortest_distance

    def update_pheromones(self, paths):
        pheromone_delta = np.zeros_like(self.pheromones)
        for path in paths:
            distance = self.calculate_distance(path)
            for i in range(len(path) - 1):
                current_city = path[i]
                next_city = path[i + 1]
                pheromone_delta[current_city, next_city] += 1 / distance
                pheromone_delta[next_city, current_city] += 1 / distance
        self.pheromones = (1 - self.evaporation_rate) * self.pheromones + pheromone_delta


    def calculate_distance(self, path):
        distance = 0
        for i in range(len(path) - 1):
            current_city = path[i]
            next_city = path[i + 1]
            distance += self.distances[current_city, next_city]
        return distance
        
    def generate_paths(self):
        paths = []
        for _ in range(self.num_ants):
            path = self.generate_path(0)
            paths.append(path)
        return paths
        
    def generate_path(self, start_city):
        # path = [start_city] 
        path = [np.random.randint(0, self.num_cities)]
        # Add cities according to probability
        while len(path) < self.num_cities:
            next_city = self.choose_next_city(path)
            path.append(next_city)
        return path
        
    def choose_next_city(self, path):
        cur_city = path[-1]
        unvisited_cities = [city for city in range(self.num_cities) if city not in path]
        prob = [self.calculate_prob(cur_city, city) for city in unvisited_cities]
        total_prob = sum(prob)
        normalized_prob = [p / total_prob for p in prob]
        next_city = np.random.choice(unvisited_cities, p=normalized_prob)
        return next_city
        
    def calculate_prob(self, cur_city, next_city):
        pheromone = self.pheromones[cur_city, next_city]
        distance = self.distances[cur_city, next_city]
        total = 0
        for city in range(self.num_cities):
            if city != cur_city:
                total += (self.pheromones[cur_city, city] ** self.alpha) * (1 / self.distances[cur_city, city] ** self.beta)
        prob = (pheromone ** self.alpha) * (1 / distance ** self.beta) / total
        return prob

In [30]:
num_iterations = 50
distances_10 = generate_distances(10)
distances = np.zeros(shape=(4, 4))
distances[0, 1] = distances[1, 0] = 10
distances[0, 2] = distances[2, 0] = 15
distances[0, 3] = distances[3, 0] = 20
distances[1, 3] = distances[3, 1] = 25
distances[2, 3] = distances[3, 2] = 30
distances[1, 2] = distances[2, 1] = 35

num_agents = [1, 5, 10, 20]
for agent in num_agents:
    colony = AntColony(distances_10, agent, num_iterations)
    shortest_path, shortest_distance = colony.run()
    print(shortest_path, shortest_distance)

[1, 2, 3, 4, 0, 5, 6, 9, 7, 8, 1] 122.0
[4, 3, 2, 1, 8, 7, 9, 6, 5, 0, 4] 122.0
[9, 6, 5, 0, 4, 3, 2, 1, 8, 7, 9] 122.0
[9, 7, 8, 1, 2, 3, 4, 0, 5, 6, 9] 122.0


In [31]:
num_iterations = 50
distances_20 = generate_distances(20)
num_agents = [1, 5, 10, 20]
for agent in num_agents:
    colony = AntColony(distances_20, agent, num_iterations)
    shortest_path, shortest_distance = colony.run()
    print(shortest_path, shortest_distance)

[10, 7, 11, 9, 8, 18, 19, 3, 14, 13, 1, 4, 2, 17, 0, 6, 12, 16, 5, 15, 10] 155.0
[7, 10, 15, 5, 16, 8, 18, 19, 3, 14, 13, 1, 4, 2, 17, 0, 6, 12, 9, 11, 7] 145.0
[17, 2, 4, 10, 15, 5, 16, 6, 12, 8, 18, 19, 3, 7, 11, 9, 1, 13, 14, 0, 17] 144.0
[3, 19, 18, 8, 12, 6, 16, 5, 15, 10, 4, 2, 17, 0, 14, 13, 1, 9, 11, 7, 3] 144.0
